# Rename ALL crawled files with random index

In [15]:
import random

random_filenames = random.sample(list(range(1, 1001)), 1000)
random_filenames = [str(i).zfill(4) for i in random_filenames]

In [13]:
import os

folder_path = './data/raw_data/'

index = 0
for filename in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, filename)):
        new_filename = random_filenames[index] + '.py'
        index += 1
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))


IndexError: list index out of range

# Load in all data and make a .tsv file

In [18]:
import os
import csv

data_folder_path = './data/raw_data/'
tsv_path = './data/dirty_data.tsv'
table = []

for filename in os.listdir(data_folder_path):
    if os.path.isfile(os.path.join(data_folder_path, filename)):
        entry = {}
        file_id = filename.split("-")[0]
        with open(data_folder_path + filename, "r") as f:
            content = f.read()
        entry = {
            "id" : file_id,
            "filename" : filename,
            "content" : content
        }
        table.append(entry)

fieldnames = table[0].keys()
with open(tsv_path, 'w', newline='') as f:
    writer = csv.DictWriter(f, delimiter='\t', fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(table)

## Read back the data to test implementation

In [23]:
data = []

with open(tsv_path, 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        data.append(row)

print(data[1]) # data[0] will be the header

['JY321', 'JY321-rstu_api_test.py', 'import pytest\nimport pytest_asyncio\n\nfrom rtsu_students_bot.rtsu import RTSUApi\n\npytest_plugins = (\'pytest_asyncio\',)\n\nTEST_DATA = {\n    "login": "your login",\n    "password": "your pass",\n}\n\n\n@pytest_asyncio.fixture()\nasync def rtsu_client():\n    """\n    Initializes client\n    :return: Prepared `RTSUApi` client\n    """\n\n    async with RTSUApi() as api:\n        yield api\n\n\n@pytest.mark.asyncio\nasync def test_rtsu_login(rtsu_client: RTSUApi):\n    """\n    Tests rtsu login\n    :param rtsu_client: A RTSU API client\n    :return:\n    """\n\n    resp = await rtsu_client.auth(TEST_DATA.get("login"), TEST_DATA.get("password"))\n\n    assert resp.token is not None\n\n\n@pytest.mark.asyncio\nasync def test_rtsu_profile_fetching(rtsu_client: RTSUApi):\n    """\n    Tests rtsu profile fetching\n    :param rtsu_client:\n    :return:\n    """\n\n    await rtsu_client.auth(TEST_DATA.get("login"), TEST_DATA.get("password"))\n\n    pro

# Load in Adjudicated labels

In [35]:
adjudicated_labels = {}

with open("./data/adjudicated_labels.txt", "r") as f:
    lines = f.readlines()

for line in lines:
    id, label = line.split()
    adjudicated_labels[id] = label

len(adjudicated_labels)

500

# Make `adjudicated.txt`

In [36]:
import os
import csv

data_folder_path = './data/final_data/'
tsv_path = './deliverables/adjudicated.txt'
table = []

for filename in os.listdir(data_folder_path):
    if filename == ".DS_Store":
        continue
    if os.path.isfile(os.path.join(data_folder_path, filename)):
        # print(filename)
        file_id = str(int(filename.split(".")[0]))
        with open(data_folder_path + filename, "r") as f:
            file_content = f.read()
        entry = [file_id, "adjudicated", adjudicated_labels[file_id], file_content]
        table.append(entry)

with open(tsv_path, 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(table)

# Load in Evaluation Labels

In [37]:
evaluation_labels = {}

with open("./data/evaluation_labels.txt", "r") as f:
    lines = f.readlines()

for line in lines:
    id, label1, label2 = line.split()
    evaluation_labels[id] = (label1, label2)

len(evaluation_labels)

250

# Make `individual_annotations.txt`

In [39]:
import os
import csv

data_folder_path = './data/final_data/'
tsv_path = './deliverables/individual_annotations.txt'
table = []

for filename in os.listdir(data_folder_path):
    if filename == ".DS_Store":
        continue
    if os.path.isfile(os.path.join(data_folder_path, filename)):
        file_id = str(int(filename.split(".")[0]))
        if file_id not in evaluation_labels:
            continue
        with open(data_folder_path + filename, "r") as f:
            file_content = f.read()
        entry1 = [file_id, "jackson", evaluation_labels[file_id][0], file_content]
        entry2 = [file_id, "donghui", evaluation_labels[file_id][1], file_content]
        table.append(entry1)
        table.append(entry2)

with open(tsv_path, 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(table)